In [4]:
#installation of VectorBT 
!pip install vectorbt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
#installation of Yfinance 
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:

#importing necessary libraries 
import vectorbt as vbt
import pandas as pd
import numpy as np
import yfinance as yf
import datetime

SystemError: ignored

In [ ]:
# Fetch historical data
# end_time = datetime.datetime.now()
# start_time = end_time - datetime.timedelta(days=2)

symbol = 'BTC-USD', 'QBIT-USD', 'DGTX-USD', 'BTG-USD', 'LINK-USD', 'ETH-USD', 'BNB-USD', 'XRP-USD', 'ADA-USD', 'DOGE-USD', 'SOL-USD', 'LTC-USD', 'TRX-USD', 'ATOM-USD', 'TON-USD', 'XMR-USD', 'OKB-USD', 'BCH-USD'
start_date = '2023-01-01'
end_date = '2023-02-11'
ohlcv = yf.download(symbol, interval="5m", period="60d")
price = ohlcv['Close']
high_price = ohlcv['High']
low_price = ohlcv['Low']

In [ ]:
print(ohlcv)

In [ ]:

# Define the RSI indicator
def rsi(price_set, rsi_window=14):
        delta = price_set.diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(rsi_window).mean()
        avg_loss = loss.rolling(rsi_window).mean()
        rs = avg_gain / avg_loss
        return 100 - (100 / (1 + rs))


# Define Divergence 
def divergence(price_set, rsi_window=14, fast_ma_window = 1, slow_ma_window = 50):
   

# Calculate the RSI indicator for using in calculation of divergence 
   rsi_indicator = rsi(price_set)
#   rsi = vbt.RSI.run(price, window = rsi_window).rsi.to_numpy()

# Было: 
   # bullish_divergence = ((ohlcv['Close'].diff() < 0) & (rsi.diff() > 0))

# Стало:
   bullish_divergence = ((price_set.diff()<0) & (rsi_indicator.diff() > 0))
   

   bearish_divergence = ((price_set.diff()<0) & (rsi_indicator.diff() < 0))
   #print((ohlcv['Close']).diff()<0)
   
   # Get fast Moving Average (MA) of the prices
   fast_ma = vbt.MA.run(price_set, fast_ma_window, short_name='fast')
   print(type(bearish_divergence))
   # Get slow Moving Average (MA) of the prices
   slow_ma = vbt.MA.run(price_set, slow_ma_window, short_name='slow')
   print(type(slow_ma))

   # Determine UpTrend (fast MA above slow MA)
   trend_up = fast_ma.ma_above(slow_ma)   

   # Determine DownTrend (fast MA below slow MA)
   trend_down = fast_ma.ma_below(slow_ma)

   # Convert to Numpy array 
   trend_up = trend_up.to_numpy()
   trend_down = trend_down.to_numpy()
   bullish_divergence = bullish_divergence.to_numpy()
   bearish_divergence = bearish_divergence.to_numpy()
   print(type(bearish_divergence))

   # Combine all previous indicators into the one 
   # Нужно, чтобы если bullish_divergence (trend_up), True ПЛЮС fast MA above slow MA, то ставим 1, все остальное - 0. Если же bearish_divergence True, ПЛЮС fast MA above slow MA (trend_down), то ставим -1
   indicator = np.where( ( ( (bullish_divergence) == True)  & (trend_up == True) ), 1, 0)
   indicator = np.where( ( ( (bearish_divergence) == True)  & (trend_down == True) ), -1, indicator)
   

   #indicator = np.where( (bullish_divergence == True), 1, 0)
   #indicator = np.where( (bearish_divergence == True), -1, indicator)
   

   #print(bullish_divergence)
   #print(bearish_divergence)
   #return bullish_divergence
   #return bearish_divergbullish_divergence.to_numpy()ence
   return indicator


# divergence(price_set = price)
#price = price.to_numpy()
indicator_divergence_ma = vbt.IndicatorFactory(
   class_name = 'Combination_Divergence_MA',
   short_name = 'DIVER_MA',
   input_names = ['price'],
   param_names = ['rsi_window', 'fast_ma_window', 'slow_ma_window'],
   output_names = ['value'],
   ).from_apply_func(
       divergence,
       rsi_window = 14, 
       fast_ma_window = 1,
       slow_ma_window = 50,
       keep_pd = True
       )
# 

print(price)

results = indicator_divergence_ma.run(
   price,
   rsi_window = 14,   
   fast_ma_window = 1,
   slow_ma_window = 50
   )

entries = results.value == 1.0
exits = results.value == -1.0

pf = vbt.Portfolio.from_signals(price, entries, exits, fees=0.01)
print(pf.total_return())
print(pf.stats())


#print(results.value)


#(divergence.bullish_divergence)


# Calculate the bullish and bearish divergence signals
#bullish_divergence = ((ohlcv['Close'].diff() < 0) & (rsi.diff() > 0))
#bearish_divergence = ((ohlcv['Close'].diff() > 0) & (rsi.diff() < 0))

   # convert bullish_divergence to numpy array 
#bullish_divergence = bullish_divergence.to_numpy()

In [ ]:
rsi = vbt.RSI.run(price, window = 14).rsi.to_numpy()
bearish_divergence = ((ohlcv['Close'].diff() < 0) & (rsi.diff() > 0))
bullish_divergence = ((ohlcv['Close'].diff() < 0) & (rsi.diff() > 0))

print(bearish_divergence)

In [ ]:
rsi = vbt.RSI.run(price, window = 14).rsi
   # Get ATR
atr = vbt.ATR.run(high_price, low_price, price)
   # adjust RSI via ATR
rsi = rsi-atr.atr
   # convert RSI to numpy array 
#rsi = rsi.to_numpy()

# Calculate the bullish and bearish divergence signals
bullish_divergence = ((ohlcv['Close'].diff() < 0) & (rsi.diff() > 0))
bearish_divergence = ((ohlcv['Close'].diff() > 0) & (rsi.diff() < 0))

   # convert bullish_divergence to numpy array 
#bullish_divergence = bullish_divergence.to_numpy()



# Get Moving Average 
fast_ma = vbt.MA.run(price, 10)
slow_ma = vbt.MA.run(price, 50)

# Determine uptrend (If price above Moving Average)
uptrend = vbt.MA.ma_above(price, slow_ma)
downtrend = slow_ma.ma.ma_above(fast_ma)
print(uptrend)
print("above is uptrend")

def custom_indicator(close, ma_window = 50, rsi_window = 33):  

   ma = vbt.MA.run(close, ma_window).ma.to_numpy()

#   atr = atr.atr.to_numpy()
   # multiplicator = 7.000000
#   rsi_adjusted_by_atr = rsi-atr.atr #* multiplicator
# Calculate the bullish and bearish divergence signals
#   bullish_divergence = ((ohlcv['Close'].diff() < 0) & (rsi_adjusted_by_atr.diff() > 0))
#   bearish_divergence = ((ohlcv['Close'].diff() > 0) & (rsi_adjusted_by_atr.diff() < 0))

   trend = np.where(rsi > 70, -1, 0)
   trend = np.where(rsi < 30, 1, trend)
   return trend

# Giving recipe to Indicator Factory for our custom indicator 
ind = vbt.IndicatorFactory(
   class_name = "Combination",
   short_name = "comb",
   input_names = ["close"],
   param_names = ["rsi_window", "ma_window"],
   output_names = ["value"]
   ).from_apply_func(
      custom_indicator,
      rsi_window = 14,
      ma_window = 50,
      )
res = ind.run(
   price,
   rsi_window = 21,
   ma_window = 50,
)

print (res.value)

In [ ]:

def combine_indicators(close, window = 14, entry = 30, exit = 70, slow = 50, fast = 10):
     trend = np.full(close.shape, np.nan)
     rsi = vbt.RSI.run(close, window = window, short_name="rsi")
     #print(rsi.rsi)
   # Get ATR
     atr = vbt.ATR.run(high_price, low_price, price)
   # adjust RSI via ATR
     rsi = rsi.rsi-atr.atr
     rsi = rsi.to_numpy()
     slow_ma = vbt.MA.run(close, slow)
     fast_ma = vbt.MA.run(close, fast)
     ma_signal = fast_ma.ma_above(slow_ma).to_numpy()
     rsi_above = rsi.rsi_above(exit).to_numpy()
     rsi_below = rsi.rsi_below(entry).to_numpy()
     for x in range(len(close)):
         if rsi_above[x] == True and ma_signal[x] == True:
             trend[x] = 1
         elif rsi_below[x] == True or ma_signal[x] == False:
             trend[x] = -1
         else:
             trend[x] = 0
     return trend


myInd = vbt.IndicatorFactory(
 class_name="Combination",
 short_name="comb",
 input_names=["close"],
 param_names=["window","entry","exit","slow","fast"],
 output_names=["ind"]
 ).from_apply_func(
 combine_indicators,
 window=14,
 entry=30,
 exit=70,
 slow=50,
 fast=10
 )

res = myInd.run(
 price,
 window=14,
 entry = 30,
 exit = 70,
 slow = 50,
 fast = 10,
 param_product=True,
 )

entries = res.ind == 1.0
exits = res.ind == -1.0

pf = vbt.Portfolio.from_signals(price, entries, exits, freq = "5m", fees=0.01)
print(pf.stats())